In [ ]:
import pandas as pd
from stockfish import Stockfish
from tqdm import tqdm
import ast

In [ ]:
tqdm.pandas(desc="Processing...", unit="game")

In [ ]:
df = pd.read_csv('../../dados/base/chess_games_chesscom.csv')

In [ ]:
df['move_evals'] = df['move_evals'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
stockfish_path = '../testes/stockfish/stockfish-windows-x86-64-avx2.exe'

params = {
    "Threads": 4,
    "Hash": 2048,
}

stockfish = Stockfish(path=stockfish_path, depth=15, parameters=params)

In [ ]:
df = df[df['player'] == 'Mr-Barros']
# df = df[df['rules'] == 'chess']

In [ ]:
def cut_eco_name(eco_url):
    eco_parts = eco_url.split('/')
    eco_full_name = eco_parts[-1]

    eco = ' '.join(eco_full_name.split('-')[:2])

    if '.' in eco:
        eco = eco.split('.')[0]
    return eco

In [ ]:
df['eco'] = df['eco'].apply(cut_eco_name)

In [ ]:
winrates = {}
for opening in df['eco'].unique():
    kings_pawn_games = df[df['eco'].str.contains(opening)]
    wins = kings_pawn_games[kings_pawn_games['player_pieces'] == kings_pawn_games['winner']]
    if len(kings_pawn_games) < len(df)/100:
        winrates[opening] = 'unknown'
    else:
        winrates[opening] = len(wins)/len(kings_pawn_games)

In [ ]:
def decode(n):

    if not isinstance(n,str):
        return

    T = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?{~}(^)[_]@#$,./&-*++="

    result = []
    w = len(n)
    for i in range(0, w, 2):
        move = {}
        drop = False
        o = T.index(n[i])
        s = T.index(n[i + 1])
        
        # Check if `s` indicates a promotion
        if s > 63:
            promotion_piece = "qnrbkp"[(s - 64) // 3]
            move["promotion"] = promotion_piece
            s = o + (-8 if o < 16 else 8) + ((s - 1) % 3) - 1

        # Check if `o` indicates a drop
        if o > 75:
            move["drop"] = "qnrbkp"[o - 79]
            drop = True
        else:
            move["from"] = T[o % 8] + str((o // 8) + 1)

        move["to"] = T[s % 8] + str((s // 8) + 1)

        # output formatting
        if not drop:
            move_string = move['from'] + move['to']
            if 'promotion' in move.keys():
                move_string += move['promotion']
            result.append(move_string)
        else:
            move_string = 'D' + move['drop'] + move['to']
            result.append(move_string)
    
    return result

# function to encode into TCN
# def encode(n):

#     if not isinstance(n,str):
#         return

#     T = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?{~}(^)[_]@#$,./&-*++="
#     if not isinstance(n, list):
#         n = [n]
        
#     result = ""
#     for move in n:
#         if "drop" in move:
#             s = 79 + "qnrbkp".index(move["drop"])
#         else:
#             s = T.index(move["from"][0]) + 8 * (int(move["from"][1]) - 1)
        
#         _ = T.index(move["to"][0]) + 8 * (int(move["to"][1]) - 1)
        
#         if "promotion" in move:
#             promotion_piece_index = "qnrbkp".index(move["promotion"])
#             _ = 3 * promotion_piece_index + 64 + (9 + _ - s if _ < s else _ - s - 7)
        
#         result += T[s] + T[_]
    
#     return result

In [ ]:
df['move_list'] = df['tcn'].apply(decode)

In [ ]:
initial_board = {
    'a8': 'r', 'b8': 'n', 'c8': 'b', 'd8': 'q', 'e8': 'k', 'f8': 'b', 'g8': 'n', 'h8': 'r',
    'a7': 'p', 'b7': 'p', 'c7': 'p', 'd7': 'p', 'e7': 'p', 'f7': 'p', 'g7': 'p', 'h7': 'p',
    'a2': 'P', 'b2': 'P', 'c2': 'P', 'd2': 'P', 'e2': 'P', 'f2': 'P', 'g2': 'P', 'h2': 'P',
    'a1': 'R', 'b1': 'N', 'c1': 'B', 'd1': 'Q', 'e1': 'K', 'f1': 'B', 'g1': 'N', 'h1': 'R'
}

piece_values = {'p': 1, 'P': 1, 'n': 3, 'N': 3, 'b': 3, 'B': 3, 'r': 5, 'R': 5, 'q': 9, 'Q': 9, 'k': 0, 'K': 0}

def count_material(move_list):
    '''
    não leva em consideração enpassant por considerações de eficiência.
    '''
    if not isinstance(move_list, list):
        return
    
    board = initial_board.copy()
    material_history = []

    for move in move_list:

        if move[0] == 'D':
            return

        from_square = move[:2]
        to_square = move[2:4]

        if len(move) > 4: # promotes the piece
            piece = board.get(from_square)
            if piece.isupper():
                board[from_square] = move[-1]
            else:
                board[from_square] = move[-1].lower()

        # Move the piece
        piece = board.get(from_square)
        if piece:
            board[to_square] = piece  # Place piece on new square
            del board[from_square]  # Remove from old square

        # Update material counts after the move
        white_material = sum(piece_values[piece] for piece in board.values() if piece.isupper())
        black_material = sum(piece_values[piece] for piece in board.values() if piece.islower())
        material_history.append((white_material, black_material))

    return material_history

In [ ]:
starting_fen ='rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
def get_game_eval(move_list):
    '''
    usa o stockfish para avaliar cada posição de um jogo.
    '''
    if not isinstance(move_list, list):
        return
    move_evals = []
    stockfish.set_fen_position(starting_fen)

    try:
        for move in move_list:
            
            # Using Stockfish to get evaluation
            stockfish.make_moves_from_current_position([move])
            eval = stockfish.get_evaluation()
            move_evals.append(eval)
        return move_evals
    except:
        return

In [ ]:
def extract_from_moves(move_list):
    """
    Function to aggregate the different extraction function loops in a single one, for the sake of efficiency.
    """
    if not isinstance(move_list, list):
        return
    

In [ ]:
df['material_count'] = df['move_list'].progress_apply(count_material)

In [ ]:
reeval = False

In [ ]:
if ('move_evals' not in df.columns) or reeval:
    df['move_evals'] = df['move_list'].progress_apply(get_game_eval)


In [ ]:
def map_eval(eval):
    if eval['type'] == 'mate':
        if eval['value'] > 0:
            return 4
        else:
            return -4
    else:
        value = eval['value']
        if value < -500:
            return -3
        elif -500 <= value < -300:
            return -2
        elif -300 <= value < -50:
            return -1
        elif -50 <= value < 50:
            return 0
        elif 50 <= value < 300:
            return 1
        elif 300 <= value < 500:
            return 2
        else:
            return 3

In [ ]:
def extrai_eval_fases(row):
    
    evals = row['move_evals']
    material_counts = row['material_count']

    if not isinstance(evals, list) or not isinstance(material_counts, list):
        return row

    if len(evals) >= 24:
        eval = evals[23]
    else:
        eval = evals[-1]
    row['opening_eval'] = map_eval(eval)

    # Getting the 
    for i, counts in enumerate(material_counts):
        if counts[0] <= 15 and counts[1] <= 15:
            eval = evals[i]
            row['midgame_eval'] = map_eval(eval)
            break

    if 'midgame_eval' not in row.index:
        eval = evals[-1]
        row['midgame_eval'] = map_eval(eval)
    elif pd.isna(row['midgame_eval']):
        eval = evals[-1]
        row['midgame_eval'] = map_eval(eval)
    
    return row

In [ ]:
def evaluate_quality(eval: dict, previous_eval: dict, turn: bool):
    WHITE = 0
    BLACK = 1

    # From centipawn to centipawn
    if previous_eval['type'] == 'cp' and eval['type'] == 'cp':
        # Assuming the player will never make the evaluation increase, we calculate how much it decreased in absolute value
        decrease = abs(eval['value'] - previous_eval['value'])

        if decrease == 0:
            quality = 'Best'
        elif decrease < 50:
            quality = 'Excellent'
        elif decrease < 100: 
            quality = 'Good'
        elif decrease < 200:
            quality = 'Inaccuracy'
        elif decrease < 300:
            quality = 'Mistake'
        else:
            quality = 'Blunder'

    # Mate sequence changed to centipawn, a mate was missed
    elif previous_eval['type'] == 'mate' and eval['type'] == 'cp':
        quality = 'Missed Mate'
    
    # Centipawn changed to mate
    # Either one player blundered a mate, or stockfish had a high centipawn eval but hadn't seen the mate sequence
    elif previous_eval['type'] == 'cp' and eval['type'] == 'mate':
        if (turn == WHITE and eval['value'] < 0) or (turn == BLACK and eval['value'] > 0):
            quality = 'Blunder' # We assume eval will never go from centipawn to mate in 0
        else:
            quality = 'Best'

    else: # both evals are of type mate
        distance_change = eval['value'] - previous_eval['value']

        if turn == WHITE:
            pass
        # else: # Black's turn
        #     distance_change


In [ ]:
# def extract_move_qualities(row):
#     evals = row['move_evals']
#     previous_eval = {

#     }

In [ ]:
original_columns = list(df.columns)
if 'opening_eval' not in original_columns:
    new_columns = ['opening_eval', 'midgame_eval']
    final_columns = original_columns + new_columns
else:
    final_columns = original_columns

df = df.apply(extrai_eval_fases, axis=1)

In [ ]:
df = df[final_columns]

In [ ]:
# add the winrates with the opening used in each game

df['winrate_with_opening'] = 0.0

for op in df['eco'].unique():
    df.loc[df['eco']==op,'winrate_with_opening'] = winrates[op]

In [ ]:
df.to_csv('../../dados/base/chess_games_chesscom.csv',index=False)